In [6]:
import pandas as pd
from pathlib import Path
from src.data_processing import convert_age, encode_dataset

In [7]:
DATASETS_PATH = Path('datasets')
train_dataset_path = DATASETS_PATH / 'train.csv'

In [8]:
data = pd.read_csv(train_dataset_path, index_col=0)
data.head()

,Grade,Case_ID,Gender,Age_at_diagnosis,Primary_Diagnosis,Race,IDH1,TP53,ATRX,PTEN,...,FUBP1,RB1,NOTCH1,BCOR,CSMD3,SMARCA4,GRIN2A,IDH2,FAT4,PDGFRA
0,GBM,TCGA-06-A6S0,Male,79 years 183 days,Glioblastoma,black or african american,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,...,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED
1,LGG,TCGA-HT-8106,Male,53 years 197 days,"Astrocytoma, anaplastic",white,MUTATED,MUTATED,NOT_MUTATED,NOT_MUTATED,...,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,MUTATED,NOT_MUTATED
2,LGG,TCGA-HT-7607,Female,61 years 305 days,"Astrocytoma, NOS",white,MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,...,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED
3,LGG,TCGA-QH-A6X5,Female,58 years 55 days,Mixed glioma,white,MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,...,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED
4,LGG,TCGA-P5-A5F1,Male,30 years 113 days,"Astrocytoma, NOS",white,MUTATED,MUTATED,MUTATED,NOT_MUTATED,...,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED,NOT_MUTATED


All columns are objects. All columns are categorical but *Age_at_diagnosis* so they should be encoded.

# Data processing

In [24]:
data_processing = data.copy().drop(['Case_ID', 'Primary_Diagnosis'], axis=1)


data_processing['Age_at_diagnosis'] = convert_age(data_processing['Age_at_diagnosis'])
data_processing = data_processing.dropna(axis=0)

object_columns = list(data_processing.select_dtypes(include='object').columns)
data_processing = encode_dataset(data_processing, columns=object_columns)
data_processing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 599 entries, 0 to 602
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Grade             599 non-null    int64  
 1   Gender            599 non-null    int64  
 2   Age_at_diagnosis  599 non-null    float64
 3   Race              599 non-null    int64  
 4   IDH1              599 non-null    int64  
 5   TP53              599 non-null    int64  
 6   ATRX              599 non-null    int64  
 7   PTEN              599 non-null    int64  
 8   EGFR              599 non-null    int64  
 9   CIC               599 non-null    int64  
 10  MUC16             599 non-null    int64  
 11  PIK3CA            599 non-null    int64  
 12  NF1               599 non-null    int64  
 13  PIK3R1            599 non-null    int64  
 14  FUBP1             599 non-null    int64  
 15  RB1               599 non-null    int64  
 16  NOTCH1            599 non-null    int64  
 17  BC

In [25]:
data_processing

,Grade,Gender,Age_at_diagnosis,Race,IDH1,TP53,ATRX,PTEN,EGFR,CIC,...,FUBP1,RB1,NOTCH1,BCOR,CSMD3,SMARCA4,GRIN2A,IDH2,FAT4,PDGFRA
0,0,1,79.501370,2,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,53.539726,4,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
2,1,0,61.835616,4,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,0,58.150685,4,0,1,1,1,1,0,...,1,1,1,0,1,1,1,1,1,1
4,1,1,30.309589,4,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,1,0,31.021918,2,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
599,1,0,27.846575,4,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
600,1,1,29.087671,4,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
601,1,1,51.854795,4,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
